In [ ]:
import pandas as pd
import re
import math
import numpy as np

In [ ]:
# Import entries data
data = pd.read_csv("entries.csv") 

In [ ]:
len(data)
data

In [ ]:
data_croho = data['Croho']

In [ ]:
data_croho.head()

In [ ]:
# Creating a link table that works as a lookup table.
linktable = data[['CrohoNo','Croho','Degree']]

linktable = linktable.drop_duplicates()

# Combine 2 columns for better mapping on Degree and Croho
linktable['Croho'] = linktable['Degree'] + " " + linktable['Croho']

linktable = linktable.dropna(subset =["CrohoNo"])
linktable

In [ ]:
data['Croho'] = data['Degree'] + " " + data['Croho'] 

# data = data.fillna(0)
# # Code that checks if the existing CrohoNo is blank and if it is blank gives it an id if there is a matching Croho 
# for index, row in data.iterrows():
#     if row['CrohoNo'] == 0:
#         if linktable.loc[linktable['Croho'] == row['Croho']].empty == False:
#             data.iloc[index,3] = linktable.loc[linktable['Croho'] == row['Croho']]['CrohoNo'].iloc[0]
# # Splits string again
# for index, row in data.iterrows():
#     data.iloc[index,2] = data.iloc[index,2].split(" ", 1)[1]

In [ ]:
data = data.replace(0,np.nan)
data.isnull().sum()

In [ ]:
data

In [ ]:
# Import koppelbestand
df = pd.read_excel("koppelbestand 2.xlsx")

In [ ]:
df

In [ ]:
df['CrohoNaamActueel']

In [ ]:
# Split Ad/M/B from CrohoNaamActueel. To link the CrohoNaamActueel data to Croho
x = 0
array = []
for i in range(len(df)):
    string = df.iloc[x,2]
    stringtest = string.split(' ',1)[0]
    string = string.split(' ',1)[1]
    array.append(stringtest)
    df.iloc[x,1] = string
    x = x +1    

In [ ]:
df['Degree'] = array
df.rename(columns={'CrohoNaamActueel':'Croho'}, inplace=True)

In [ ]:
df

In [ ]:
df3 = pd.merge(data, df, how='outer', on=['Croho','Degree','Jaar'])
df_test_duplicates = df3
df3['Unnamed: 0']
df3

In [ ]:
df3 = df3[['Unnamed: 0', 'Croho','Jaar', 'CrohoNo','OpleidingsVorm','Opmerking','Degree','CrohoActueel','Locatie','Label1','Label2']]
df3 = df3.drop_duplicates()
df3

In [ ]:
df3.isnull().sum()

In [ ]:
df3.dropna(subset =["Unnamed: 0"])

In [ ]:
df3.to_csv('Raw_data_combined.csv')

In [ ]:
# Changing column types and picking a set few amount of columns from the entry data
# df.columns = ['CrohoActueel','Croho','InstituteID','Institute','Locatie']
df.columns = ['Jaar','CrohoActueel','Croho','Locatie','InstituteID','Institute','Degree']
df["InstituteID"] = df["InstituteID"].round().astype(int)
df["CrohoActueel"] = df["CrohoActueel"].round().astype(int)

In [ ]:
# Creating a talbe for Institute/Croho
Institutedf = df[['InstituteID','Institute']]
Institutedf = Institutedf.drop_duplicates(subset='InstituteID', keep="last")
Crohodf = df[['CrohoActueel','Croho','InstituteID','Locatie','Degree']]
Crohodf

In [ ]:
data
# For loop to take out double spaces
x = 0 
for i in range(len(data)):
    string = data.iloc[x,2]
    string = re.sub(' +', ' ',string)
    data.iloc[x,2] = string
    x = x +1 
    print(string)

In [ ]:
# Merging entry dataset with koppelbestand dataset
df3 = pd.merge(data, df, how='outer', on=['Croho','Degree'])
df_test_duplicates = df3
#df3 = df3.loc[~df3['Unnamed: 0'].duplicated(keep='first')]
df_test_duplicates.to_csv('test_duplicate_merging_data.csv')
df3.to_csv('Raw_data_combined.csv')
len(df3)

In [ ]:
df3.isnull().sum()

In [ ]:
newblankdf = df3

In [ ]:
newblankdf

In [ ]:
math.isnan(newblankdf.iloc[1,3])
print(newblankdf.iloc[1,3])

In [ ]:
# If ChrohoActueel is blank it checks if ChrohoNo is blank or not and proceeds to replace the ChrohoActueel value.
x = 0 
x1 = 0
x2 = 0
for i in range(len(newblankdf)):
    print(math.isnan(newblankdf.iloc[x,9]))
    if math.isnan(newblankdf.iloc[x,9]):
        if math.isnan(newblankdf.iloc[x,3]):
            newblankdf.iloc[x,9] = float('nan')
            x2 = x2 + 1
        else:
            newblankdf.iloc[x,9] = newblankdf.iloc[x,3]
            #print(newblankdf.iloc[x,9])
            x1 = x1 +1 
    x = x +1

In [ ]:
print(x1)

In [ ]:
print(newblankdf.isnull().sum())
len(newblankdf)

In [ ]:
namechangesdf = newblankdf[['Unnamed: 0','Brin','Croho','CrohoNo','Jaar','OpleidingsVorm','Opmerking','Taal','Degree','CrohoActueel','InstituteID']]


In [ ]:
namechangesdf = namechangesdf[np.isfinite(namechangesdf['CrohoActueel'])]
print(namechangesdf.isnull().sum())
len(namechangesdf)

In [ ]:
entries = namechangesdf[['Unnamed: 0','Brin','CrohoNo','Jaar','OpleidingsVorm','Opmerking','Taal','Degree','CrohoActueel','InstituteID']]

# entries["InstituteID"] = entries["InstituteID"].round().astype(int)
entries["CrohoActueel"] = entries["CrohoActueel"].round().astype(int)

In [ ]:
entries

In [ ]:
entries['InstituteID'] = entries['InstituteID'].fillna(0)
Crohodf[Crohodf['CrohoActueel'] == 30039]

In [ ]:
x = 0
false = 0
for index, row in entries.iterrows():
    if row['InstituteID'] == 0:
        if Crohodf.loc[Crohodf['InstituteID'] == row['InstituteID']].empty:
            print(row['CrohoActueel'])
            if (row['CrohoActueel'] in Crohodf['CrohoActueel'].values) == False:
                print('False')
                false = false +1 
            else:
                # data.iloc[index,3] = linktable.loc[linktable['Croho'] == row['Croho']]['CrohoNo'].iloc[0]
                print(Crohodf.loc[Crohodf['CrohoActueel'] == row['CrohoActueel']]['InstituteID'].iloc[0])
                entries.iloc[x,9] = Crohodf.loc[Crohodf['CrohoActueel'] == row['CrohoActueel']]['InstituteID'].iloc[0]
                print(f'After  {entries.iloc[index,9]}/{entries.iloc[index,8]}')
                array.append({entries.iloc[index,8]})
        # print(x)
        x = x+1
print(x)

In [ ]:
df = pd.DataFrame(data=array)
print(false)

In [ ]:
df

In [ ]:
entries.to_csv('Entries-1-noblank.csv')
# newblankdf.to_csv('Entries-1.csv')
Institutedf.to_csv('Institute_map.csv')
Crohodf.to_csv('Croho_map.csv')

# Getting old names

# Catching as many name changes as possible
# These name changes were encountered in excel, whene exploring the data
namechangesdf = newblankdf 

#namechangesdf['CrohoActueel'].isnull() == 1    

namechangesdf

# Creating a link table that works as a lookup table.
linktable = data[['CrohoNo','Croho','Degree']]

linktable = linktable.drop_duplicates()

# Combine 2 columns for better mapping on Degree and Croho
linktable['Croho'] = linktable['Degree'] + " " + linktable['Croho']

linktable = linktable.dropna(subset =["CrohoNo"])

linktable

linktable = linktable.drop_duplicates()

linktable['Croho'] = linktable['Degree'] + " " + linktable['Croho']

linktable

linktable = linktable.dropna(subset =["CrohoNo"])

linktable.loc[linktable['Croho'] == 'Communicatie']

data['Croho'] = data['Degree'] + " " + data['Croho'] 

data = data.fillna(0)

data

data = data.fillna(0)
# Code that checks if the existing CrohoNo is blank and if it is blank gives it an id if there is a matching Croho 
for index, row in data.iterrows():
    if row['CrohoNo'] == 0:
        if linktable.loc[linktable['Croho'] == row['Croho']].empty == False:
            data.iloc[index,3] = linktable.loc[linktable['Croho'] == row['Croho']]['CrohoNo'].iloc[0]
# Splits string again
for index, row in data.iterrows():
    data.iloc[index,2] = data.iloc[index,2].split(" ", 1)[1]

# Splits string again
for index, row in data.iterrows():
    data.iloc[index,2] = data.iloc[index,2].split(" ", 1)[1]

data.loc[data['Croho'] == 'Accountancy']

data.to_csv('Data_incsv.csv')
linktable.to_csv('Linktable-Test.csv')